In [1]:
import requests
import json
import pandas as pd
import sqlite3

In [2]:
stub_url = "https://pokeapi.co/api/v2/ability/"

In [3]:
# read sqlite pokemon id and name into pandas dataframe
con = sqlite3.connect("pokemonGo.sqlite")
df_names = pd.read_sql_query("SELECT * from pokemon_names", con)

con.close()

In [4]:
df_names.head()

,id,pokename
0,1,Bulbasaur
1,2,Ivysaur
2,3,Venusaur
3,4,Charmander
4,5,Charmeleon


In [5]:
id_list = df_names["id"].tolist()
poke_id_list = []
url_list = []
desc_list = []

In [6]:
for i in range(890):
    try:
        poke_id_list.append(str(id_list[i]))
        url_list.append(stub_url + str(id_list[i]))
        url_response = requests.get(f"{url_list[i]}").json()
        desc_list.append(url_response["effect_entries"][1]["effect"])
    except:
        desc_list.append("None")

In [7]:
complete_list = [id_list,url_list,desc_list]

df_desc = pd.DataFrame(complete_list).transpose()
df_desc.columns = ['id','url','desc']
df_desc = df_desc.dropna()
df_desc = df_desc[df_desc["desc"] != "None"]
df_desc

,id,url,desc
0,1,https://pokeapi.co/api/v2/ability/1,This Pokémon's damaging moves have a 10% chanc...
1,2,https://pokeapi.co/api/v2/ability/2,The weather changes to rain when this Pokémon ...
2,3,https://pokeapi.co/api/v2/ability/3,This Pokémon's Speed rises one stage after eac...
3,4,https://pokeapi.co/api/v2/ability/4,Moves cannot score critical hits against this ...
4,5,https://pokeapi.co/api/v2/ability/5,"When this Pokémon is at full HP, any hit that ..."
...,...,...,...
186,187,https://pokeapi.co/api/v2/ability/187,Strengthens fairy moves for all friendly and o...
187,188,https://pokeapi.co/api/v2/ability/188,"While this Pokémon is on the field, dark aura ..."
188,189,https://pokeapi.co/api/v2/ability/189,When this Pokémon enters battle or gains this ...
189,190,https://pokeapi.co/api/v2/ability/190,When this Pokémon enters battle or gains this ...


In [10]:
df_name_desc = pd.merge(df_names, df_desc,how="inner",on="id")
df_name_desc

,id,pokename,url,desc
0,1,Bulbasaur,https://pokeapi.co/api/v2/ability/1,This Pokémon's damaging moves have a 10% chanc...
1,2,Ivysaur,https://pokeapi.co/api/v2/ability/2,The weather changes to rain when this Pokémon ...
2,3,Venusaur,https://pokeapi.co/api/v2/ability/3,This Pokémon's Speed rises one stage after eac...
3,4,Charmander,https://pokeapi.co/api/v2/ability/4,Moves cannot score critical hits against this ...
4,5,Charmeleon,https://pokeapi.co/api/v2/ability/5,"When this Pokémon is at full HP, any hit that ..."
...,...,...,...,...
186,187,Hoppip,https://pokeapi.co/api/v2/ability/187,Strengthens fairy moves for all friendly and o...
187,188,Skiploom,https://pokeapi.co/api/v2/ability/188,"While this Pokémon is on the field, dark aura ..."
188,189,Jumpluff,https://pokeapi.co/api/v2/ability/189,When this Pokémon enters battle or gains this ...
189,190,Aipom,https://pokeapi.co/api/v2/ability/190,When this Pokémon enters battle or gains this ...


# Insert New DataFrame into SQL Lite as Database Table

In [11]:
conn = sqlite3.connect('pokemonGo.sqlite')
c = conn.cursor()

In [12]:
try:
    c.execute('DROP TABLE pokemon_name_desc')
except:
    pass
conn.commit()

In [13]:
try:
    #create db table to hold pokemon id and names
    c.execute('''CREATE TABLE pokemon_name_desc(
        id integer PRIMARY KEY,
        pokename text,
        url text,
        desc text
    )''')
except:
    pass
conn.commit()

In [14]:
#append dataframe to db table
df_name_desc.to_sql("pokemon_name_desc",conn,if_exists='append',index=False)
conn.close()